In [ ]:
!pip install split-folders

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay, classification_report
from tensorflow.keras.models import model_from_json

import splitfolders

In [ ]:
splitfolders.ratio("../input/dtf-vigilia", output="./data", seed=1000, ratio=(.85, .15))

In [ ]:
TRAIN_DIR = './data/train'
TEST_DIR = './data/val'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

color_mode="rgb"  #rgb       grayscale
channels = 3
width = 1706
height = 1165
batch_size = 25

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

test_datagen = ImageDataGenerator(
        rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    color_mode=color_mode,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, # same directory as training data
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    color_mode=color_mode,
    subset='validation') # set as validation data


test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                   target_size=(height, width), 
                                                   batch_size = 1, 
                                                   shuffle=False,
                                                   color_mode=color_mode,
                                                   class_mode="binary")

In [ ]:
# load json and create model
json_file = open('../input/dtf-vigilia-weights/weights/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

model.summary()

model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# Loads the weights
model.load_weights('../input/dtf-vigilia-weights/weights/weights-110.hdf5')

In [ ]:
def evaluate_set(set_generator):
    model.evaluate(set_generator)
    
    pred =  model.predict(set_generator)
    y_pred = [1 * (x[0]>=0.5) for x in pred]

    conf = confusion_matrix(set_generator.classes, y_pred)

    disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels=np.unique(set_generator.labels))

    disp = disp.plot(cmap=plt.cm.Blues, xticks_rotation='0')
    plt.show()
    
    print(classification_report(set_generator.classes, y_pred))
    
    TP = conf[1][1]
    TN = conf[0][0]
    FP = conf[0][1]
    FN = conf[1][0]
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    print('accuracy:', round(conf_accuracy,4))
    conf_specificity = (TN / float(TN + FP))
    print('specificity:', round(conf_specificity,4))
    conf_sensitivity = (TP / float(TP + FN))
    print('sensitivity:', round(conf_sensitivity,4))
    conf_precision = (TP / float(TP + FP))
    print('precision:', round(conf_precision,4))

In [ ]:
evaluate_set(train_generator)

In [ ]:
evaluate_set(validation_generator) 

In [ ]:
evaluate_set(test_generator) 